In [52]:
import pandas as pd
import matplotlib.pyplot as plt

url = "DAT_ASCII_AUDJPY_M1_2018.csv"
df = pd.read_csv(url, delimiter = ';', header = None, names = ["Data", "Open", "High", "Low", "Close", "Volume"], parse_dates = ["Data"])
df.set_index("Data", inplace = True) 
df.index.names = [None]
df = df.drop("Volume", axis=1)                                                    
df.index = df.index.tz_localize('EST').tz_convert('Brazil/West').tz_localize(None)

# Aplicando condicionais nos dados (Cor da vela(1 - Verde, -1 - Vermelho, 0 - Cinza))

df.loc[(df.Close > df.Open), 'Color'] = 1
df.loc[(df.Close < df.Open), 'Color'] = -1
df.loc[(df.Close == df.Open), 'Color'] = 0

#Cor de entrada (MHI)

df.loc[((df.Color.eq(0).rolling(3).sum()).shift(1) >= 1), 'MHI'] = 0
df.loc[((df.Color.eq(1).rolling(3).sum()).shift(1) >= 2), 'MHI'] = 1
df.loc[((df.Color.eq(-1).rolling(3).sum()).shift(1) >= 2), 'MHI'] = -1

#Programando as estratégias (1 - WIN, -1 LOSS, 0 SEM OPERAÇÃO)
#MHI
#WINS&LOSS

df.loc[(df.MHI == df.Color), 'Result'] = 1
df.loc[(df.MHI != df.Color), 'Result'] = -1

#SEM OPERAÇÃO
df.loc[(df.MHI == 0), 'Result'] = 0
df.loc[(df.MHI.isnull()), 'Result'] = 0

df[:15]


,Open,High,Low,Close,Color,MHI,Result
2018-01-01 18:00:00,87.917,87.917,87.917,87.917,0.0,NaN,0.0
2018-01-01 18:02:00,87.904,87.904,87.904,87.904,0.0,NaN,0.0
2018-01-01 18:03:00,87.902,87.902,87.897,87.897,-1.0,NaN,0.0
2018-01-01 18:04:00,87.894,87.894,87.884,87.884,-1.0,0.0,0.0
2018-01-01 18:05:00,87.884,87.901,87.825,87.840,-1.0,-1.0,1.0
2018-01-01 18:06:00,87.836,87.869,87.836,87.843,1.0,-1.0,-1.0
2018-01-01 18:07:00,87.840,87.863,87.840,87.863,1.0,-1.0,-1.0
2018-01-01 18:08:00,87.861,87.861,87.854,87.854,-1.0,1.0,-1.0
2018-01-01 18:09:00,87.854,87.855,87.836,87.836,-1.0,1.0,-1.0
2018-01-01 18:10:00,87.835,87.847,87.835,87.847,1.0,-1.0,-1.0


In [33]:
#MHI 1

op = df.iloc[::5, :].drop("Open", axis=1).drop("High", axis=1).drop("Low", axis=1).drop("Close", axis=1)

#MÉDIA MOVEL (WINRATE)

op["Winrate"] = ((op["Result"].eq(1).rolling(24).sum())/
                 ((op["Result"].eq(1).rolling(24).sum())+(op["Result"].eq(-1).rolling(24).sum()))) * 100

op.head()

,Color,MHI,Winrate
2018-01-01 18:00:00,0.0,NaN,NaN
2018-01-01 18:06:00,1.0,-1.0,NaN
2018-01-01 18:11:00,-1.0,1.0,NaN
2018-01-01 18:16:00,-1.0,-1.0,NaN
2018-01-01 18:21:00,1.0,1.0,NaN


In [38]:
# #Plot Winrate x Tempo
# ax = op['Winrate'][100:3000].plot(figsize = (20,10))
# ax.axhline(50, color='xkcd:dark grey', alpha=0.6)
# ax.axhline(73, color='xkcd:dark grey', alpha=0.6)
# plt.show()

In [ ]:
#Análise de ponto de corte de Winrate com melhor resultado (WINRATE = X -> ENTRADA NA PRÓXIMA VELA)

WINRATE / WIN / LOSS / %W / %L